In [1]:
import torch
import numpy as np
from torch import nn
from utils.model import Spice_model
from torchinfo import summary
import pandas as pd
from data_files.dataloader import MedleyDBLoader
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

from utils.model import Spice_model
from utils.training_script import Trainer
from optims.loss import Huber_loss, Recons_loss, Conf_loss
from data_files.dataset import CQT_Dataset

2023-07-23 13:36:34.200920: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-23 13:36:34.420058: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-23 13:36:35.164033: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
fs = 16000
med = MedleyDBLoader(fs )

In [5]:
f1 = 1110.0644999999954 
f2 = -5.684341886080802e-14
dive = f1/f2
dive

-1.9528461205301544e+16

In [6]:
np.log2(dive)

/tmp/ipykernel_23855/3868652844.py:1: RuntimeWarning: invalid value encountered in log2
  np.log2(dive)


nan

In [3]:
data_pd = pd.read_pickle('./CQT_data/MIR1k.pkl')

In [6]:
nnp = np.load('./CQT_data/MedleyDB.npy')

In [ ]:
nnp = torch.toTe(nnp)

In [4]:
data_pd.shape

(3530, 191)

In [7]:
#data_pd = pd.DataFrame(data=nnp) 
train, val = train_test_split(data_pd, train_size=0.8, test_size=0.2, random_state=1)
print("train shape: ", train.shape)
train_batches = DataLoader(CQT_Dataset(data=train, mode='train'), batch_size=64, shuffle=True)
print("train_batch NUmber: ", len(train_batches))
diff, slice1, slice2, f0 = next(iter(train_batches))
print(f"diff batch shape: {diff.size()}")
print(f"slice1 batch shape: {slice1.size()}")
print(f"slice2 batch shape: {f0.size()}")

spice = Spice_model()


for b in train_batches:
    print("batch shape:", b[0].shape)
    pitch_diff, x_1, x_2, f0 = b
    x_1 = x_1.type(torch.FloatTensor)
    #print(x_1.shape, x_1.type())
    a, x, y = spice(x_1)
    print(a.size(), x.size(), y.size())

train shape:  (2824, 191)
train_batch NUmber:  45
diff batch shape: torch.Size([64])
slice1 batch shape: torch.Size([64, 128])
slice2 batch shape: torch.Size([64])
batch shape: torch.Size([64])
torch.Size([64, 1]) torch.Size([64, 1]) torch.Size([64, 32, 4])
batch shape: torch.Size([64])
torch.Size([64, 1]) torch.Size([64, 1]) torch.Size([64, 32, 4])
batch shape: torch.Size([64])
torch.Size([64, 1]) torch.Size([64, 1]) torch.Size([64, 32, 4])
batch shape: torch.Size([64])
torch.Size([64, 1]) torch.Size([64, 1]) torch.Size([64, 32, 4])
batch shape: torch.Size([64])
torch.Size([64, 1]) torch.Size([64, 1]) torch.Size([64, 32, 4])
batch shape: torch.Size([64])
torch.Size([64, 1]) torch.Size([64, 1]) torch.Size([64, 32, 4])
batch shape: torch.Size([64])
torch.Size([64, 1]) torch.Size([64, 1]) torch.Size([64, 32, 4])
batch shape: torch.Size([64])
torch.Size([64, 1]) torch.Size([64, 1]) torch.Size([64, 32, 4])
batch shape: torch.Size([64])
torch.Size([64, 1]) torch.Size([64, 1]) torch.Size([64

In [6]:
xx=torch.arange(0, 25, 1)

In [12]:
xx = xx.reshape((5,-1))

In [13]:
xx.size()

torch.Size([5, 5])

In [ ]:
df = pd.read_pickle('./CQT_data/MIR1k.pkl')

In [ ]:
df.shape

In [ ]:
np.arange(0,25,1).reshape(5,5)[:,-1:]

In [ ]:
a = np.arange(0,15,1).reshape(5,3)
a = np.hstack((a, np.array([0.25,0.7,0,0.5,0]).reshape(-1,1)))
a

In [ ]:
df = pd.DataFrame(a)
df.shape

In [ ]:
# remove rows of cqt where label (last) column is zero
df.drop(df.loc[df.iloc[:, -1]==0].index, inplace=True)
df.shape

In [ ]:
np.min(df.iloc[:, -1])

In [14]:
s = Spice_model()
summary(s, input_size=[10, 128])

Encoder forward torch.FloatTensor
Encoder forward torch.FloatTensor
Encoder forward torch.FloatTensor
Encoder forward torch.FloatTensor
Encoder forward torch.FloatTensor
Encoder forward torch.FloatTensor
no unpooling True True
no unpooling False True
no unpooling False True
no unpooling False True
no unpooling False True


Layer (type:depth-idx)                   Output Shape              Param #
Spice_model                              [10, 1]                   --
├─Spice_Encoder: 1-1                     [10, 1]                   --
│    └─Conv_block: 2-1                   [10, 64, 64]              --
│    │    └─Conv1d: 3-1                  [10, 64, 128]             256
│    │    └─BatchNorm1d: 3-2             [10, 64, 128]             128
│    │    └─ReLU: 3-3                    [10, 64, 128]             --
│    │    └─MaxPool1d: 3-4               [10, 64, 64]              --
│    └─Conv_block: 2-2                   [10, 128, 32]             --
│    │    └─Conv1d: 3-5                  [10, 128, 64]             24,704
│    │    └─BatchNorm1d: 3-6             [10, 128, 64]             256
│    │    └─ReLU: 3-7                    [10, 128, 64]             --
│    │    └─MaxPool1d: 3-8               [10, 128, 32]             --
│    └─Conv_block: 2-3                   [10, 256, 16]             --
│    │  

In [24]:
for n, param in s.named_parameters():
    print(type(param), param.requires_grad)

<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Parameter'> True
<class 'torch.nn.parameter.Param